In [8]:
import pandas as pd
import numpy as np
from sklearn import metrics as sklearn_metrics


In [13]:
min_label = 0.
max_label = 4.

def min_max_norm(x, min_label=min_label, max_label=max_label):
    return (x-min_label)/(max_label-min_label)

def reverse_min_max_norm(x, min_label=min_label, max_label=max_label):
    return x*(max_label-min_label)+min_label

In [3]:
outputs = pd.read_pickle('./checkpoints/outputs.pkl')

In [4]:
pred1, true1, pred2, true2 = outputs

In [44]:
def calibrate(logit_list):
    for i in range(len(logit_list)):
        if 0.98<=logit_list[i]<=1.03:
            logit_list[i] = 1
    return logit_list

def get_score(y_pred, y_true, pred2, true2):
    prediction_list = []
    ground_truth_list = []
    for biopsy_id in y_pred:
        preds = np.array(y_pred[biopsy_id])
        truths = np.array(y_true[biopsy_id])
        
        preds2 = np.array(pred2[biopsy_id])
        truths2 = np.array(true2[biopsy_id])
        
        final_pred1 = preds.mean()
        final_pred2 = preds2.mean()
        
        prediction_list.append((final_pred1*1.+final_pred2*1.)/2)
        ground_truth_list.append(truths.mean())
    prediction_list = np.array(prediction_list)
    ground_truth_list = np.array(ground_truth_list)
    prediction_list = reverse_min_max_norm(prediction_list)
    calibrate(prediction_list)
    ground_truth_list = reverse_min_max_norm(ground_truth_list)

    mse = sklearn_metrics.mean_squared_error(ground_truth_list, prediction_list)
    return mse

print(get_score(pred1, true1, pred2, true2))

0.5913342927999048
